In [1]:
import pandas as pd
import numpy as np
from collections import OrderedDict
from skcriteria import Data
from skcriteria.madm.simus import SIMUS
from skcriteria.madm.electre import ELECTRE1
from skcriteria.madm import closeness, simple
from scipy.stats import rankdata
from utils import *

In [2]:
df = pd.read_excel('input.xlsx') # reading input file

In [3]:
df

,Cost,Risk,FPMK
0,10,1,1.32
1,12,5,1.30
2,11,1,1.38
3,14,2,1.25


In [4]:
a = 1.0
criteria_matrix = np.array([[1.0, a],[1.0/a, 1.0]])# np.array([[1.0,a,b],[1/a,1.0,c],[1/c,1/b,1.0]])

In [5]:
criteria_matrix = np.ones((2,2))
criteria_matrix

array([[1., 1.],
       [1., 1.]])

In [14]:
criteria_matrix = np.ones((2,2))
AHP_rank(df.iloc[:,:2],criteria_matrix)

<ipython-input-12-13032c78384c>:83: RuntimeWarning: invalid value encountered in double_scalars
  CR = CI/self.RI[self.size]


Exception: Consistency is not sufficient to reach a decision

In [12]:
# AHP function
class AHP():
    RI = {1:0, 2:0.0, 3:0.58, 4:0.9, 5:1.12, 6:1.24, 7:1.32, 8:1.41, 9:1.45}
    consistency = False
    priority_vec = None
    compete = False
    normal = False
    sublayer = None

    def __init__(self,name,size):
        self.name = name
        self.size = size
        self.matrix = np.zeros([size,size])
        self.criteria = [None]*size

    def update_matrix(self,mat,automated=True):
        self.original_matrix = mat
        if not((mat.shape[0] == mat.shape[1]) and (mat.ndim == 2)):
            raise Exception('Input matrix must be squared')
        
        if self.size != len(self.criteria):
            self.criteria = [None]*self.size
        self.matrix = mat
        self.size = mat.shape[0]
        self.consistency = False
        self.normal = False
        self.priority_vec = None
        if automated:
            self.rank()
    
    def input_priority_vec(self,vec):
        if not(vec.shape[1]==1) and (vec.shape[0]==self.size) and (vec.ndim==2):
            raise Exception('Size of input priority vector is not compatible.')
        self.priority_vec = vec
        self.output = self.priority_vec/self.priority_vec.sum()
        self.consistency = True
        self.normal = True

    def rename(self,name):
        self.name = name

    def update_criteria(self,criteria):
        if len(criteria) == self.size:
            self.criteria = criteria
        else:
            raise Exception('Input does not match number of criteria')
    
    def add_layer(self,alternative):
        if not self.criteria:
            raise Exception('Please input criteria before adding new layer')
        self.compete = False
        self.sublayer = OrderedDict()
        self.alternative = alternative
        for i in range(self.size):
            self.sublayer[self.criteria[i]] = AHP(self.criteria[i],len(alternative))
            self.sublayer[self.criteria[i]].update_criteria(self.alternative)

    def normalize(self):
        if self.normal:
            pass
        self.col_sum = self.matrix.sum(axis=0)
        try:
            self.matrix = self.matrix/self.col_sum
        except:
            raise Exception('Error when normalizing on columns')
        else:
            self.normal = True
            self.priority_vec = self.matrix.sum(axis=1).reshape(-1,1)

    def rank(self):
        if self.consistency:
            df = pd.DataFrame(data = self.output, index = self.criteria, columns=[self.name])
            return df
        
        if not self.normal:
            self.normalize()
        
        Ax = self.matrix.dot(self.priority_vec)
        eigen_val = (Ax/self.priority_vec).mean()
        eigen_val = np.linalg.eig(self.original_matrix)[0].max()
        CI = (eigen_val - self.size)/(self.size-1)

        if self.size > 2:
            CR = CI/self.RI[self.size]
        else:
            CR = 0.0

        if CR<0.1:
            self.consistency = True
            self.output = self.priority_vec/self.priority_vec.sum()
            self.df_out = pd.DataFrame(data = self.output, index = self.criteria, columns = [self.name])
            return self.df_out
        else:
            raise Exception('Consistency is not sufficient to reach a decision')

    def make_decision(self):
        if not self.consistency:
            self.rank()
        if not self.compete:
            temp = True
            arrays = []
            for item in self.sublayer.values():
                item.rank()
                temp = temp and item.consistency
                if temp:
                    arrays.append(item.output)
                else:
                    raise Exception('Please check AHP for {}'.format(item.name))
            
            if temp:
                self.compete = True
            else:
                pass
            self.recommendation = np.concatenate(arrays, axis=1).dot(self.output)
        self.df_decision = pd.DataFrame(data=self.recommendation, index=self.alternative, columns = ['AHP Score'])
        self.df_decision.index.name = 'Alternative'
        self.df_decision['rank'] = self.df_decision['AHP Score'].rank(ascending=True)
        return self.df_decision

def AHP_rank(df,criteria_matrix):
    method = AHP('method',df.shape[1]) # the second argument is number of objectives
    method.update_criteria(list(df.columns))
    method.update_matrix(criteria_matrix)

    method.add_layer([i for i in range(df.shape[0])]) #adding interventions

    # iterate over column names --> method.sublayer['col_name'].input_priority_vec(insert corresponding col here)
    # finally method.make_decision
    for col in df.columns: #adding priority vectors associated with objectives corresponding to different interventions
        method.sublayer[col].input_priority_vec(np.array(df[col]).reshape(-1,1))

    return method.make_decision()#['rank']#.values#.astype(int)

# scikit-criteria methods

In [4]:
data = Data(
    # the alternative matrix
    mtx=np.array(df),
    # optimal sense
    criteria=[min, min, min],
    # names of alternatives and criteria
    anames=list(df.index),
    cnames=list(df.columns))

In [5]:
data

ALT./CRIT.,Cost (min),Risk (min),FPMK (min)
0,10,1,1
1,10,2,1.3
2,10,1,1.3
3,11,2,1.25


In [6]:
# SIMUS()
dm = SIMUS()
dec = dm.decide(data,b=[None]*df.shape[1])
# let's see the decision
dec

ALT./CRIT.,Cost (min),Risk (min),FPMK (min),Rank
0,10,1,1,1
1,10,2,1.3,2
2,10,1,1.3,3
3,11,2,1.25,4


In [7]:
# TOPSIS
dm = closeness.TOPSIS()
dec = dm.decide(data)
# let's see the decision
dec

ALT./CRIT.,Cost (min),Risk (min),FPMK (min),Rank
0,10,1,1,1
1,10,2,1.3,3
2,10,1,1.3,2
3,11,2,1.25,4


# AHP

In [19]:
# AHP function
class AHP():
    RI = {1:0, 2:0, 3:0.58, 4:0.9, 5:1.12, 6:1.24, 7:1.32, 8:1.41, 9:1.45}
    consistency = False
    priority_vec = None
    compete = False
    normal = False
    sublayer = None

    def __init__(self,name,size):
        self.name = name
        self.size = size
        self.matrix = np.zeros([size,size])
        self.criteria = [None]*size

    def update_matrix(self,mat,automated=True):
        self.original_matrix = mat
        if not((mat.shape[0] == mat.shape[1]) and (mat.ndim == 2)):
            raise Exception('Input matrix must be squared')
        
        if self.size != len(self.criteria):
            self.criteria = [None]*self.size
        self.matrix = mat
        self.size = mat.shape[0]
        self.consistency = False
        self.normal = False
        self.priority_vec = None
        if automated:
            self.rank()
    
    def input_priority_vec(self,vec):
        if not(vec.shape[1]==1) and (vec.shape[0]==self.size) and (vec.ndim==2):
            raise Exception('Size of input priority vector is not compatible.')
        self.priority_vec = vec
        self.output = self.priority_vec/self.priority_vec.sum()
        self.consistency = True
        self.normal = True

    def rename(self,name):
        self.name = name

    def update_criteria(self,criteria):
        if len(criteria) == self.size:
            self.criteria = criteria
        else:
            raise Exception('Input does not match number of criteria')
    
    def add_layer(self,alternative):
        if not self.criteria:
            raise Exception('Please input criteria before adding new layer')
        self.compete = False
        self.sublayer = OrderedDict()
        self.alternative = alternative
        for i in range(self.size):
            self.sublayer[self.criteria[i]] = AHP(self.criteria[i],len(alternative))
            self.sublayer[self.criteria[i]].update_criteria(self.alternative)

    def normalize(self):
        if self.normal:
            pass
        self.col_sum = self.matrix.sum(axis=0)
        try:
            self.matrix = self.matrix/self.col_sum
        except:
            raise Exception('Error when normalizing on columns')
        else:
            self.normal = True
            self.priority_vec = self.matrix.sum(axis=1).reshape(-1,1)

    def rank(self):
        if self.consistency:
            df = pd.DataFrame(data = self.output, index = self.criteria, columns=[self.name])
            return df
        
        if not self.normal:
            self.normalize()
        
        Ax = self.matrix.dot(self.priority_vec)
        eigen_val = (Ax/self.priority_vec).mean()
        eigen_val = np.linalg.eig(self.original_matrix)[0].max()
        CI = (eigen_val - self.size)/(self.size-1)
        CR = CI/self.RI[self.size]
        
        if CR<0.1 or (self.RI[self.size]==0):
            self.consistency = True
            self.output = self.priority_vec/self.priority_vec.sum()
            self.df_out = pd.DataFrame(data = self.output, index = self.criteria, columns = [self.name])
            return self.df_out
        else:
            raise Exception('Consistency is not sufficient to reach a decision')

    def make_decision(self):
        if not self.consistency:
            self.rank()
        if not self.compete:
            temp = True
            arrays = []
            for item in self.sublayer.values():
                item.rank()
                temp = temp and item.consistency
                if temp:
                    arrays.append(item.output)
                else:
                    raise Exception('Please check AHP for {}'.format(item.name))
            
            if temp:
                self.compete = True
            else:
                pass
            self.recommendation = np.concatenate(arrays, axis=1).dot(self.output)
        self.df_decision = pd.DataFrame(data=self.recommendation, index=self.alternative, columns = ['AHP Score'])
        self.df_decision.index.name = 'Alternative'
        self.df_decision['rank'] = self.df_decision['AHP Score'].rank(ascending=True)
        return self.df_decision

def AHP_rank(df,criteria_matrix):
    method = AHP('method',df.shape[1])
    method.update_criteria(list(df.columns))
    method.update_matrix(criteria_matrix)

    method.add_layer([i for i in range(df.shape[0])])

    # iterate over column names --> method.sublayer['col_name'].input_priority_vec(insert corresponding col here)
    # finally method.make_decision
    for col in df.columns:
        method.sublayer[col].input_priority_vec(np.array(df[col]).reshape(-1,1))

    return method.make_decision()['rank'].values

In [27]:
method = AHP('method',df.shape[1])
method.update_criteria(list(df.columns))
a = 1.0
b = 1.0
c = 1.0

criteria_matrix = np.array([[1.0,a,b],[1/a,1.0,c],[1/c,1/b,1.0]])

method.update_matrix(criteria_matrix)

method.add_layer([i for i in range(df.shape[0])])

for col in df.columns:
    method.sublayer[col].input_priority_vec(np.array(df[col]).reshape(-1,1))

method.make_decision()['rank'].values.astype(int)

In [38]:
df['Risk'].argsort().argsort()+1

0    0
1    2
2    1
3    3
Name: Risk, dtype: int64

# Sum of ranks

In [106]:
sum_of_rank = np.zeros((df.shape[0],1))
for col in df.columns:
    temp = rankdata(df[col], method='min').reshape(-1,1)
    sum_of_rank += temp
#     print(col)
#     print(np.array(df[col].argsort().argsort()+1).reshape(-1,1))
    

In [107]:
sum_of_rank

array([[3.],
       [9.],
       [6.],
       [9.]])

In [111]:
sum_of_rank.ravel().argsort().argsort()+1

array([1, 3, 2, 4], dtype=int64)

In [85]:
np.array(df['Cost'].argsort()+1).reshape(-1,1)

array([[1],
       [3],
       [2],
       [4]], dtype=int64)

In [87]:
sum_of_rank

array([[ 3.],
       [10.],
       [ 6.],
       [11.]])

# Check if Michael ranks and AHP differ?


In [23]:
def AHP_rank(df):    
    method = AHP('method',df.shape[1])
    method.update_criteria(list(df.columns))
    method.update_matrix(np.ones((3,3)))

    method.add_layer([i for i in range(df.shape[0])])

    # iterate over column names --> method.sublayer['col_name'].input_priority_vec(insert corresponding col here)
    # finally method.make_decision
    for col in df.columns:
        method.sublayer[col].input_priority_vec(np.array(df[col]).reshape(-1,1))

    return method.make_decision()['rank'].values#.astype(int)

In [38]:
def sum_of_rank(df):
    sum_of_rank = np.zeros((df.shape[0],1))
    for col in df.columns:
        temp = rankdata(df[col], method='min').reshape(-1,1)
        sum_of_rank += temp
    return sum_of_rank.ravel().argsort().argsort()+1

In [40]:
def TOPSIS_SIMUS(df):
    data = Data(
    # the alternative matrix
    mtx=np.array(df),
    # optimal sense
    criteria=[min, min, min],
    # names of alternatives and criteria
    anames=list(df.index),
    cnames=list(df.columns))
    dm = closeness.TOPSIS()
    topsis = dm.decide(data)

    # dm = SIMUS()
    # simus = dm.decide(data,b=[None]*df.shape[1])
    return topsis.rank_#,simus.rank_

In [28]:
a = AHP_rank(df).astype(int)

In [29]:
b = sum_of_rank(df)

In [31]:
np.array_equal(a,b)

True

In [42]:
TOPSIS_SIMUS(df)

array([1, 3, 2, 4])

In [43]:
AHP_rank(df_).astype(int)

array([3, 1, 2])

In [53]:
np.random.rand(3,3)

array([[0.5488135 , 0.71518937, 0.60276338],
       [0.54488318, 0.4236548 , 0.64589411],
       [0.43758721, 0.891773  , 0.96366276]])

In [61]:
for i in range(100):
    
    np.random.seed(i)    
    df_ = pd.DataFrame(np.random.randint(10,size=(4,3)))
    if np.array_equal(AHP_rank(df_).astype(int),TOPSIS_SIMUS(df_)):
        pass
    else:
        print(df_.head())
        

   0  1  2
0  8  9  3
1  8  8  0
2  5  3  9
3  9  5  7
   0  1  2
0  5  6  8
1  6  1  6
2  4  8  1
3  8  5  1
   0  1  2
0  9  4  0
1  1  9  0
2  1  8  9
3  0  8  6
   0  1  2
0  5  4  0
1  4  6  6
2  4  8  4
3  2  8  7
   0  1  2
0  5  5  4
1  7  2  5
2  1  3  9
3  7  7  1
   0  1  2
0  2  7  0
1  2  7  6
2  6  2  2
3  9  1  7
   0  1  2
0  5  6  3
1  3  5  8
2  5  7  8
3  8  3  2
   0  1  2
0  9  5  0
1  9  5  0
2  5  4  9
3  5  5  4
   0  1  2
0  5  4  0
1  2  9  7
2  6  4  9
3  7  1  8
   0  1  2
0  3  0  7
1  9  5  3
2  4  9  1
3  5  0  6
   0  1  2
0  3  5  5
1  7  9  9
2  3  6  9
3  3  7  8
   0  1  2
0  6  2  8
1  0  4  5
2  5  5  4
3  7  8  0
   0  1  2
0  6  3  2
1  3  6  7
2  7  3  1
3  5  2  4
   0  1  2
0  3  2  3
1  6  2  0
2  1  1  3
3  3  9  5
   0  1  2
0  8  0  1
1  4  5  1
2  0  5  0
3  2  1  4
   0  1  2
0  6  9  3
1  6  5  2
2  4  7  2
3  1  0  4
   0  1  2
0  1  3  9
1  8  9  8
2  2  4  5
3  4  1  7


In [62]:
df_

,0,1,2
0,1,3,9
1,8,9,8
2,2,4,5
3,4,1,7
